In [88]:
# !pip install --upgrade tweeterpy
# !pip install gspread_dataframe 

In [81]:
# !jupyter nbconvert --to script "D:/Github/TweeterPy/get_tweet_stuff.ipynb" --no-prompt

In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import Markdown
import json 
from datetime import datetime
import pandas as pd
from tweeterpy import TweeterPy
from functools import reduce 

In [4]:
twitter = TweeterPy()

2023-10-28 15:38:44,778 [ERROR] :: Couldn't get the API file Url.
'NoneType' object has no attribute 'group'
Traceback (most recent call last):
  File "d:\Github\TweeterPy\tweeterpy\api_util.py", line 68, in _get_api_file_url
    api_file_name = re.search(api_file_regex, page_source).group(1)
AttributeError: 'NoneType' object has no attribute 'group'
2023-10-28 15:38:48,478 [INFO] :: API Updated Successfully.


In [7]:
liked_tweets = twitter.get_liked_tweets('rngland')

2023-10-28 15:45:22,186 [INFO] :: User is authenticated.
2023-10-28 15:45:22,187 [INFO] :: User is authenticated.


dict

In [25]:
 # load an .py in to a dict 

# with open('liked_tweets.py', 'r') as file:
#     file_contents = file.read()
# my_dict = {}

# exec(file_contents, my_dict)
 

In [125]:
# import json
# import datetime

# def timedelta_serializer(obj):
#     if isinstance(obj, datetime.timedelta):
#         return str(obj)
#     raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

# json_str = json.dumps(liked_tweets, default=timedelta_serializer)

# with open('liked_tweets.json', 'w') as f:
#     f.write(json_str)


4611233

In [129]:
rest_ids = []
created_at = []
profile_images = []
screen_names = []
name = []
full_text = []
expanded_urls = []
binding_values = []
in_reply_to = []


for i in range(len(liked_tweets['data'])):
    
    # rest_id
    if 'rest_id' in liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']:
        rest_ids.append(liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']['rest_id'])
    
    
    # legacy
    # created_at
    if 'legacy' in liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']:
        created_at.append(datetime.strptime(liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['created_at'], "%a %b %d %H:%M:%S %z %Y").strftime("%Y-%m-%d %H:%M:%S")) 
        full_text.append(liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['full_text']) 
    # expanded_urls 
        if len(liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls']) >= 1:
            _1 = liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']['legacy']['entities']['urls']        
            _2 = [p['expanded_url'] for p in _1]
            expanded_urls.append('  '.join(_2))
        else:
            expanded_urls.append(' ')
        if "in_reply_to_status_id_str" in liked_tweets["data"][i]["content"]['itemContent']['tweet_results']['result']['legacy']:
            in_reply_to.append((liked_tweets["data"][i]["content"]['itemContent']['tweet_results']['result']['legacy']["in_reply_to_status_id_str"], liked_tweets["data"][i]["content"]['itemContent']['tweet_results']['result']['legacy']["in_reply_to_screen_name"],liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']['rest_id']))
        
    # core
    # profile_images    
    if 'core' in liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']:
        profile_images.append(liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['profile_image_url_https']) 
    # screen_names
        screen_names.append(liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['screen_name'])
    # name   
        name.append(liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']['core']['user_results']['result']['legacy']['name'])
    # binding_values
        if 'card' in liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']:
            if 'legacy' in liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']['card']:
                binding_values.append((liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']['card']['legacy']['binding_values'],liked_tweets['data'][i]['content']['itemContent']['tweet_results']['result']['rest_id']))  

    
in_reply_to = [(f"https://twitter.com/{in_reply_to[i][1]}/status/{in_reply_to[i][0]}",in_reply_to[i][2],) for i in range(len(in_reply_to))]
    

p_img = ["=IMAGE(\"" + profile_images[i] + "\"" + ',4 ,45, 45)'  for i in range(len(profile_images))]

p_img = [image.replace("'", "") for image in p_img]

profile_url =['https://twitter.com/' + i for i in screen_names]

tweet_url =['https://twitter.com/' + screen_names[i] + '/status/' + rest_ids[i] for i in range(len(
screen_names))]


field_names = [
"rest_ids",
"created_at",
"profile_url",
"p_img",
"screen_names",
"name",
"full_text",
"expanded_urls",
"tweet_url",
# "card_title",
# "card_description",
# "card_thumbnail_image",
# "card_photo_image_full_size_alt_text"
]


data_dict = {field: eval(field) for field in field_names}
df = pd.DataFrame(data_dict)


card_title = [(p['value']['string_value'],i[1]) for i in binding_values for p in i[0] if p['key'] =='title']

card_description = [(p['value']['string_value'],i[1]) for i in binding_values for p in i[0] if p['key'] =='description']

card_thumbnail_image = [(p['value']['image_value']['url'],i[1]) for i in binding_values for p in i[0] if p['key'] =='thumbnail_image']

card_thumbnail_image = [("=IMAGE(\"" + card_thumbnail_image[i][0] + "\"" + ',4 ,80, 80)', card_thumbnail_image[i][1],)  for i in range(len(card_thumbnail_image))]

card_photo_image_full_size_alt_text = [(p['value']['string_value'],i[1]) for i in binding_values for p in i[0] if p['key'] =='photo_image_full_size_alt_text']

card_list = [
    "card_title", 
"card_description", 
"card_thumbnail_image", 
# "card_photo_image_full_size_alt_text"
"in_reply_to"]

card_dict = {card_type: eval(card_type) for card_type in card_list}


df.rest_ids = df.rest_ids.astype('int64')
dataframes = [pd.DataFrame(v, columns=[f"{k}", 'id']) for k, v in card_dict.items()]
dataframes = [df.astype({'id': 'int64'}) for df in dataframes]
dataframes = [df.set_index("id") for df in dataframes]
final_df = reduce(lambda x, y: pd.merge(x, y, left_on='rest_ids', how='left', right_index=True), dataframes, df)



field_names = [
"rest_ids",
"created_at",
"profile_url",
"p_img",
"screen_names",
"name",
"full_text",
"tweet_url",
"in_reply_to",
"expanded_urls",
"card_title",
"card_description",
"card_thumbnail_image",
#   "card_photo_image_full_size_alt_text",
]

final_df = final_df.reindex(columns=field_names)

[f"{i} :  {len(eval(i))}" for i in field_names]

final_df



['rest_ids :  730',
 'created_at :  730',
 'profile_url :  730',
 'p_img :  730',
 'screen_names :  730',
 'name :  730',
 'full_text :  730',
 'tweet_url :  730',
 'in_reply_to :  166',
 'expanded_urls :  730',
 'card_title :  134',
 'card_description :  126',
 'card_thumbnail_image :  98']

,rest_ids,created_at,profile_url,p_img,screen_names,name,full_text,tweet_url,in_reply_to,expanded_urls,card_title,card_description,card_thumbnail_image
0,1717113723477713171,2023-10-25 09:39:48,https://twitter.com/predict_addict,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",predict_addict,"Valeriy M., PhD, MBA, CQF",@ahmedtakrouri It produces miscalibrated class...,https://twitter.com/predict_addict/status/1717...,https://twitter.com/ahmedtakrouri/status/17171...,https://medium.com/towards-data-science/python...,Python’s «predict_proba» Doesn’t Actually Pred...,Data scientists typically evaluate their predi...,"=IMAGE(""https://pbs.twimg.com/card_img/1717222..."
1,1717673764224385145,2023-10-26 22:45:12,https://twitter.com/lavitalenta,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",lavitalenta,Lord of Leisure,You’re passing out candy on Halloween? I’m air...,https://twitter.com/lavitalenta/status/1717673...,NaN,,NaN,NaN,NaN
2,1717190591103303888,2023-10-25 14:45:15,https://twitter.com/thepatwalls,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",thepatwalls,Pat Walls,Million dollar business idea hiding in plain s...,https://twitter.com/thepatwalls/status/1717190...,NaN,http://excelformulabot.com,NaN,NaN,NaN
3,1716677855554175417,2023-10-24 04:47:49,https://twitter.com/Scearpo,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",Scearpo,Scorched Earth Policy,There are 1 million holocausts occurring in yo...,https://twitter.com/Scearpo/status/17166778555...,NaN,,NaN,NaN,NaN
4,1716718321977176413,2023-10-24 07:28:37,https://twitter.com/main_horse,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",main_horse,main,@yacineMTB the cure to being a mathlet is sitt...,https://twitter.com/main_horse/status/17167183...,https://twitter.com/yacineMTB/status/171671569...,https://www.youtube.com/playlist?list=PLXqc0KM...,Diffusion Models Study Group,Diffusion Model Study Group hosted in EleutherAI,"=IMAGE(""https://pbs.twimg.com/card_img/1716718..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,1588191925521326084,2022-11-03 15:30:36,https://twitter.com/danzn1,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",danzn1,danzn1,I published plottable to PyPI! 🎉\n\nYou can no...,https://twitter.com/danzn1/status/158819192552...,NaN,https://plottable.readthedocs.io/en/latest/ h...,NaN,NaN,NaN
726,1587867700231081987,2022-11-02 18:02:15,https://twitter.com/NoahTopper,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",NoahTopper,Noah Topper 🔍⏸️,I’m reading “What Computers Still Can’t Do” an...,https://twitter.com/NoahTopper/status/15878677...,NaN,,NaN,NaN,NaN
727,1584659361632161792,2022-10-24 21:33:28,https://twitter.com/radshaan,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",radshaan,Ishan,Every single talented engineer (electrical/sof...,https://twitter.com/radshaan/status/1584659361...,NaN,,NaN,NaN,NaN
728,1499838567111143431,2022-03-04 20:05:54,https://twitter.com/tyba_latam,"=IMAGE(""https://pbs.twimg.com/profile_images/1...",tyba_latam,tyba,"¡Invierte tu dinero, no juegues con él! tyba, ...",https://twitter.com/tyba_latam/status/14998385...,NaN,,NaN,NaN,NaN


In [84]:
# import gspread
# from gspread_dataframe import set_with_dataframe

# credentials = {
#     "installed": {
#         "client_id": "117044127800-dg6q9197854uc0g375ul2d7569d8v9mj.apps.googleusercontent.com",
#         "project_id": "docs-382418",
#         "auth_uri": "https://accounts.google.com/o/oauth2/auth",
#         "token_uri": "https://oauth2.googleapis.com/token",
#         "auth_provider_x509_cert_url": "https://www.googleapis.com/oauth2/v1/certs",
#         "client_secret": "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",
#         "redirect_uris": ["http://localhost"],
#     }
# }

In [85]:
# gc, authorized_user = gspread.oauth_from_dict(credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=117044127800-dg6q9197854uc0g375ul2d7569d8v9mj.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A54354%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&state=r4K5xD0gQHS9e0R64ZJVir4teKMZWi&access_type=offline
2023-10-28 16:27:14,298 [INFO] :: "GET /?state=r4K5xD0gQHS9e0R64ZJVir4teKMZWi&code=4/0AfJohXlWm5nUSIjbo5o2PMeWcv3App1ZZ2nleSJh0VqEDM9n5-UczbM5KcuzcXK-fIQtoA&scope=https://www.googleapis.com/auth/drive%20https://www.googleapis.com/auth/spreadsheets HTTP/1.1" 200 65


In [130]:
sh = gc.open("liked_tweets")

worksheet  = sh.get_worksheet(0)    

worksheet.format(f"g2:g{len(rest_ids)}", {
  "textFormat": {
    "fontSize": 14
   }
})
set_with_dataframe(worksheet, final_df) 

{'spreadsheetId': '1vVIUT1HnLsaAGIRRJbJqI6Z7kVdmaLwUd7Ei-NkPPHA',
 'replies': [{}]}